# Test zur Inzidenzberechnung

Daten gedownloaded am 07.12.2021 zum Main-Kinzig-Kreis

Aktuelle Angaben im Dashboard:
7-Tage-Inzidenz: 321,1 (Fälle*100.000/Einwohner)
7-Tage-Fallzahl: 1.354

(Also betrachtete Einwohner: 1.354*100.000/321,1 = 421.675)

In [74]:
import pandas as pd

In [75]:
import datetime
def change_date(df):
    for _id, row in df.iterrows():
        date_s = row['Meldedatum'].split()[0]
        date = datetime.datetime.strptime(date_s, '%Y/%m/%d').date()
        df.at[_id,'Meldedatum'] = date
    return df

In [76]:
df = pd.read_csv("RKI_COVID19.csv")
df = df.drop(columns=['IdBundesland','Bundesland','Landkreis','IdLandkreis','ObjectId'])
df = change_date(df)
df.head()

,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
0,A05-A14,M,1,0,2021-03-19,"07.12.2021, 00:00 Uhr",0,-9,2021/03/19 00:00:00+00,0,1,1,Nicht übermittelt
1,A00-A04,W,3,0,2021-03-05,"07.12.2021, 00:00 Uhr",0,-9,2021/03/05 00:00:00+00,0,3,0,Nicht übermittelt
2,A05-A14,M,2,0,2021-03-19,"07.12.2021, 00:00 Uhr",0,-9,2021/03/19 00:00:00+00,0,2,0,Nicht übermittelt
3,A00-A04,W,1,0,2021-03-06,"07.12.2021, 00:00 Uhr",0,-9,2021/03/03 00:00:00+00,0,1,1,Nicht übermittelt
4,A05-A14,M,2,0,2021-03-20,"07.12.2021, 00:00 Uhr",0,-9,2021/03/17 00:00:00+00,0,2,1,Nicht übermittelt


## Richtiger Zeitrahmen (Meldedatum 30.11. - 07.12.)

Man muss also eigtl 8 Tage betrachten!

In [82]:
start_date = datetime.date(2021,11,30)
end_date = datetime.date(2021,12,7)

In [83]:
mask = (df['Meldedatum'] >= start_date) & (df['Meldedatum'] <= end_date)
df = df.loc[mask]
df

,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
699,A00-A04,W,1,0,2021-11-30,"07.12.2021, 00:00 Uhr",0,-9,2021/11/28 00:00:00+00,-9,0,1,Nicht übermittelt
700,A00-A04,W,3,0,2021-11-30,"07.12.2021, 00:00 Uhr",0,-9,2021/11/30 00:00:00+00,-9,0,0,Nicht übermittelt
701,A00-A04,W,1,0,2021-12-01,"07.12.2021, 00:00 Uhr",0,-9,2021/11/27 00:00:00+00,-9,0,1,Nicht übermittelt
702,A00-A04,W,4,0,2021-12-01,"07.12.2021, 00:00 Uhr",0,-9,2021/12/01 00:00:00+00,-9,0,0,Nicht übermittelt
703,A00-A04,W,1,0,2021-12-02,"07.12.2021, 00:00 Uhr",0,-9,2021/12/02 00:00:00+00,-9,0,1,Nicht übermittelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16600,A80+,W,1,0,2021-12-06,"07.12.2021, 00:00 Uhr",1,-9,2021/12/03 00:00:00+00,-9,0,1,Nicht übermittelt
16601,A80+,W,3,0,2021-12-06,"07.12.2021, 00:00 Uhr",1,-9,2021/12/06 00:00:00+00,-9,0,0,Nicht übermittelt
16607,unbekannt,M,1,0,2021-12-03,"07.12.2021, 00:00 Uhr",0,-9,2021/12/03 00:00:00+00,-9,0,0,Nicht übermittelt
16608,unbekannt,unbekannt,1,0,2021-12-02,"07.12.2021, 00:00 Uhr",0,-9,2021/12/02 00:00:00+00,-9,0,0,Nicht übermittelt


## Summe Fälle

In [84]:
summe = 0
for _id, row in df.iterrows():
    if row['NeuerFall'] in (0, 1):
        summe += row['AnzahlFall']
summe

1354

--> Richtiges Ergebnis!